In [1]:
import cobra
from cobra.medium import minimal_medium
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import numpy as np
import pandas as pd

In [2]:
def set_minimal_media(model,ions=[],auto_ions=True,ignore =[]):
    ''' set media to minimal (M9) FBA (no carbon source) by default it provides a predeterimined set of ions (
    auto_ions =True), you can specificy which metaoblites not to change for example if you have calibrated the 
    vmaxes or want anaerobic conditions'''
    if auto_ions == True:
        ions = ions + ['ca2_e', 'cl_e','cobalt2_e','cu2_e','fe2_e','fe3_e','h_e','h2o_e','k_e','mg2_e','mn2_e',
                       'mobd_e','na1_e','nh4_e','ni2_e',
                            'o2_e','pi_e','so4_e','zn2_e',
                            'tungs_e','sel_e','slnt_e','cbl1_e']
    exch = [x for x in model.reactions  if len(x.metabolites) == 1 and next(iter(x.metabolites.values())) == -1 and next(iter(x.metabolites.keys())).compartment == 'e']
    #If compartments are not asigned to the model 
    if len(exch) ==0:
        exch = [x for x in model.reactions  if len(x.metabolites) == 1 and next(iter(x.metabolites.values())) == -1 and next(iter(x.metabolites.keys())).id[-2:] == '_e']

    # .. media
    for l in exch:
        if next(iter(l.metabolites.keys())).id in ions:
            model.reactions.get_by_id(l.id).lower_bound = -1000
        elif next(iter(l.metabolites.keys())).id not in ignore:
            model.reactions.get_by_id(l.id).lower_bound = -0
        else:
            pass

### The Old version  -ion limited

In [3]:
'''load the base model for REL606.'''
base_model = cobra.io.read_sbml_model('iECB_1328.xml.gz')
max_growth = base_model.slim_optimize()
base_model.medium = minimal_medium(base_model,max_growth)

''' set glucose to 1.0. This is to set relative molarity of citrate and glucose in the media properly,
so that growth yields of Cit- on DM25 and Cit+ on DM0 and DM25 are "apples to apples". '''
citminus_medium = base_model.medium
citminus_medium['EX_glc__D_e'] = 1.0
base_model.medium = citminus_medium
print(base_model.medium)

Academic license - for non-commercial use only
{'EX_ca2_e': 0.005113800273256573, 'EX_cl_e': 0.005113800273256573, 'EX_cobalt2_e': 2.4561960966650205e-05, 'EX_cu2_e': 0.0006965772130141999, 'EX_fe2_e': 0.015779586203414755, 'EX_glc__D_e': 1.0, 'EX_k_e': 0.19177291387853412, 'EX_mg2_e': 0.00852300045542762, 'EX_mn2_e': 0.0006788926011182116, 'EX_mobd_e': 0.00012673971858791504, 'EX_nh4_e': 10.61157669982634, 'EX_ni2_e': 0.0003173405356891206, 'EX_o2_e': 17.57432917782036, 'EX_pi_e': 0.9477291587675234, 'EX_so4_e': 0.24779088701595373, 'EX_zn2_e': 0.00033502514758510876}


### The New version  -ions unlimited

In [29]:
'''load the base model for REL606.'''
base_model = cobra.io.read_sbml_model('iECB_1328.xml.gz')
base_model.reactions.ATPM.lower_bound = 0.0
set_minimal_media(base_model)
''' set glucose to 1.0. This is to set relative molarity of citrate and glucose in the media properly,
so that growth yields of Cit- on DM25 and Cit+ on DM0 and DM25 are "apples to apples". '''
citminus_medium = base_model.medium
citminus_medium['EX_glc__D_e'] = 1.0
base_model.medium = citminus_medium
print(base_model.medium)

{'EX_ca2_e': 1000, 'EX_cbl1_e': 1000, 'EX_cl_e': 1000, 'EX_cobalt2_e': 1000, 'EX_cu2_e': 1000, 'EX_fe2_e': 1000, 'EX_fe3_e': 1000, 'EX_glc__D_e': 1.0, 'EX_h_e': 1000, 'EX_h2o_e': 1000, 'EX_k_e': 1000, 'EX_mg2_e': 1000, 'EX_mn2_e': 1000, 'EX_mobd_e': 1000, 'EX_na1_e': 1000, 'EX_nh4_e': 1000, 'EX_ni2_e': 1000, 'EX_o2_e': 1000, 'EX_pi_e': 1000, 'EX_sel_e': 1000, 'EX_slnt_e': 1000, 'EX_so4_e': 1000, 'EX_tungs_e': 1000, 'EX_zn2_e': 1000}


In [30]:
''' add this reaction to maeA: nad[c] + oaa[c] ->pyr[c] + nadh[c].
    This reaction is the difference between base_model and maeA_model.'''

maeA_side_reaction = cobra.Reaction('MAEA')
maeA_side_reaction.name = 'maeA_side_rxn '
maeA_side_reaction.subsystem = 'Anaplerotic Reactions'
maeA_side_reaction.lower_bound = 0.  # This is the default
maeA_side_reaction.upper_bound = 1000.  # This is the default

maeA_model = base_model.copy()
maeA_model.add_reaction(maeA_side_reaction)

''' 
cite: 
https://www.uniprot.org/uniprot/C3TAB2
https://en.wikipedia.org/wiki/Malate_dehydrogenase_(oxaloacetate-decarboxylating)
https://enzyme.expasy.org/EC/1.1.1.38
'''

maeA_side_reaction.add_metabolites({
    'oaa_c': -1.0,
    'pyr_c': 1.0,
    'co2_e': 1.0,
})

maeA_side_reaction.reaction

Read LP format model from file /tmp/tmpuzfxpu92.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros


'oaa_c --> co2_e + pyr_c'

In [31]:
''' TODO: USE CITROBACTER AS A CONTROL
    PROBLEM: KBase and BiGG database IDs are not compatible!
    Have to figure out how to do the mapping.. so skip this for now. 
    load the curated Citrobacter model (Cuepas et al. 2016)
    use this model as a positive control to compare flux distribution.'''
citrobacter_model = cobra.io.read_sbml_model('C.sedlakii_MAP_gapfill.sbml')
citro_max_growth = citrobacter_model.slim_optimize()
''' give citrobacter the same media as base'''
base_model.medium
#citrobacter_model.medium = minimal_medium(base_model, max_growth)

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00060_c0 "L_Methionine_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00067_c0 "H_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00001_c0 "H2O_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00035_c0 "L_Alanine_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11590_c0 "met_L_ala_L_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00161_c0 "L_Threonine_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11582_c0 "ala_L_Thr_L_c0">
Use of the sp

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15682_c0 "1_2_diisohexadecanoyl_sn_glycerol_3_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15676_c0 "1_isohexadecanoyl_sn_glycerol_3_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11441_c0 "fa6coa_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11493_c0 "ACP_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15328_c0 "1_octadec_11_enoyl_sn_glycerol_3_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15527_c0 "1_2_dioctadec_11_enoyl_sn_glycerol_3_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11825_c0 "Octadecenoyl_ACP_c0">
Use of the species charge attribute is discouraged, use fbc:charge instea

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00177_c0 "dADP_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00115_c0 "dATP_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15492_c0 "kdo_phospho_heptosyl_phospho_heptosyl_heptosyl_kdo2_lipidA_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15550_c0 "phospho_heptosyl_phospho_heptosyl_heptosyl_kdo2_lipidA_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd02546_c0 "CMP_KDO_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00196_c0 "Acetylphosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00029_c0 "Acetate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd01844_c0 "Propionyl_phosphate_c0">


Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd01242_c0 "Thyminose_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd01242_e0 "Thyminose_e0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00105_c0 "D_Ribose_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00105_e0 "D_Ribose_e0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11473_c0 "2E_Hexenoyl_acp_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11479_c0 "D_3_Hydroxyhexanoyl_acp_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11467_c0 "2E_Tetradecenoyl_acp_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd11483_c0 "R_3_Hydroxyoctanoyl_acyl_carrier_protein_c0">
Use of the species charge attribute 

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd15440_c0 "Carnitinyl_CoA_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00254_e0 "Mg_e0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00254_c0 "Mg_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00149_e0 "Co2_e0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00149_c0 "Co2_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd09818_c0 "Naphthyl_2_methyl_succinic_acid_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd00078_c0 "Succinyl_CoA_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species cpd09819_c0 "Naphthyl_2_methyl_succinyl_CoA_c0">
Use of the species charge attribute is discouraged, use fbc:charge inst

...

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discour

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discour

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discour

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: EX_cpd00151_e0: cpd00151_e0 <=> cpd00151_b
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discour

{'EX_ca2_e': 1000,
 'EX_cbl1_e': 1000,
 'EX_cl_e': 1000,
 'EX_cobalt2_e': 1000,
 'EX_cu2_e': 1000,
 'EX_fe2_e': 1000,
 'EX_fe3_e': 1000,
 'EX_glc__D_e': 1.0,
 'EX_h_e': 1000,
 'EX_h2o_e': 1000,
 'EX_k_e': 1000,
 'EX_mg2_e': 1000,
 'EX_mn2_e': 1000,
 'EX_mobd_e': 1000,
 'EX_na1_e': 1000,
 'EX_nh4_e': 1000,
 'EX_ni2_e': 1000,
 'EX_o2_e': 1000,
 'EX_pi_e': 1000,
 'EX_sel_e': 1000,
 'EX_slnt_e': 1000,
 'EX_so4_e': 1000,
 'EX_tungs_e': 1000,
 'EX_zn2_e': 1000}

In [32]:
''' NOTE: This code uses BiGG model IDs (BiGG namespace).'''
def generate_models(basemodel):
    ''' Cit- model growing on glucose (no citrate flux)'''
    glucose_model = basemodel.copy()
    ''' Cit+ model on DM0 with citrate flux'''
    DM0citrate_model = basemodel.copy()
    ## set glucose to zero, and citrate to 10.0.
    DM0 = DM0citrate_model.medium
    DM0['EX_glc__D_e'] = 0.0
    DM0['EX_cit_e'] = 10.0
    DM0citrate_model.medium = DM0

    assert 'EX_cit_e' in DM0citrate_model.medium
    assert 'EX_glc__D_e' not in DM0citrate_model.medium
    assert 'EX_glc__D_e' in glucose_model.medium
    assert 'EX_cit_e' not in glucose_model.medium

    ''' Cit+ model on DM25 with citrate flux'''
    DM25citrate_model = DM0citrate_model.copy()
    DM25 = DM25citrate_model.medium
    DM25['EX_glc__D_e'] = 1.0
    DM25['EX_cit_e'] = 10.0
    DM25citrate_model.medium = DM25

    ''' now make the fermentation (no O2 influx) models.'''
    glucose_fermentation_model = glucose_model.copy()
    noO2_media = glucose_fermentation_model.medium
    noO2_media['EX_o2_e'] = 0.0
    glucose_fermentation_model.medium = noO2_media
    
    DM0citrate_fermentation_model = DM0citrate_model.copy()
    DM0_noO2_media = DM0citrate_fermentation_model.medium
    DM0_noO2_media['EX_o2_e'] = 0.0
    DM0citrate_fermentation_model.medium = DM0_noO2_media
    
    DM25citrate_fermentation_model = DM25citrate_model.copy()
    DM25_noO2_media = DM25citrate_fermentation_model.medium
    DM25_noO2_media['EX_o2_e'] = 0.0
    DM25citrate_fermentation_model.medium = DM25_noO2_media
    
    
    return (glucose_model, DM0citrate_model, DM25citrate_model,glucose_fermentation_model, DM0citrate_fermentation_model, DM25citrate_fermentation_model)

In [33]:
old_cit_minus_model, old_DM0_cit_plus_model, old_DM25_cit_plus_model,old_cit_minus_ferment_model, old_DM0_cit_plus_ferment_model, old_DM25_cit_plus_ferment_model = generate_models(base_model)

new_cit_minus_model, new_DM0_cit_plus_model, new_DM25_cit_plus_model,new_cit_minus_ferment_model, new_DM0_cit_plus_ferment_model, new_DM25_cit_plus_ferment_model = generate_models(maeA_model)

Read LP format model from file /tmp/tmpbta6s0w5.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmpf9w1u5jh.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmptvwnolrt.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmpv9i2o308.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmpd3nfujru.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmp100n_xds.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmpckc_ld0p.lp
Reading time = 0.01 seconds
: 1951 rows, 5498 columns, 21264 nonzeros
Read LP format model from file /tmp/tmpivombthh.lp
Reading time = 0.01 seconds
: 1951 rows, 5498 columns, 21264 nonzeros
Read LP format model from file /

In [34]:
old_cit_minus_solution = old_cit_minus_model.optimize()
old_DM0_cit_plus_solution = old_DM0_cit_plus_model.optimize()
old_DM25_cit_plus_solution = old_DM25_cit_plus_model.optimize()

new_cit_minus_solution = new_cit_minus_model.optimize()
new_DM0_cit_plus_solution = new_DM0_cit_plus_model.optimize()
new_DM25_cit_plus_solution = new_DM25_cit_plus_model.optimize()

In [35]:
old_cit_minus_ferment_solution = old_cit_minus_ferment_model.optimize()
old_DM0_cit_plus_ferment_solution = old_DM0_cit_plus_ferment_model.optimize()
old_DM25_cit_plus_ferment_solution = old_DM25_cit_plus_ferment_model.optimize()

new_cit_minus_ferment_solution = new_cit_minus_ferment_model.optimize()
new_DM0_cit_plus_ferment_solution = new_DM0_cit_plus_ferment_model.optimize()
new_DM25_cit_plus_ferment_solution = new_DM25_cit_plus_ferment_model.optimize()

In [36]:
''' all fermentation solutions are dead.'''
print(new_cit_minus_ferment_solution.objective_value)
print(new_DM0_cit_plus_ferment_solution.objective_value)
print(new_DM25_cit_plus_ferment_solution.objective_value)

0.027637195158161695
0.17702624054210184
0.21463732887791137


In [37]:
print(new_cit_minus_solution.objective_value)
print(new_DM0_cit_plus_solution.objective_value)
print(new_DM25_cit_plus_solution.objective_value)

0.09955465387061918
0.731175540186676
0.8365993776414389


In [13]:
''' no predicted flux through Citrate Lyase.'''
print(new_cit_minus_solution['CITL'])
print(new_DM25_cit_plus_solution['CITL'])
print(new_DM0_cit_plus_solution['CITL'])

0.0
0.0
0.03860084816545495


In [14]:
''' This is the canonical rxn catalyzed by MaeA'''
print(old_cit_minus_solution['ME1'])
print(old_DM25_cit_plus_solution['ME1'])
print(old_DM0_cit_plus_solution['ME1'])

0.0
0.0
0.0


In [15]:
print(old_cit_minus_solution['ME2'])
print(old_DM25_cit_plus_solution['ME2'])
print(old_DM0_cit_plus_solution['ME2'])

0.0
0.5416178638638556
4.354100850130325


In [16]:
''' This is the canonical rxn catalyzed by MaeA'''
print(new_cit_minus_solution['ME1'])
print(new_DM25_cit_plus_solution['ME1'])
print(new_DM0_cit_plus_solution['ME1'])

0.0
0.0
0.0


In [17]:
print(old_cit_minus_solution['ME2'])
print(old_DM25_cit_plus_solution['ME2'])
print(old_DM0_cit_plus_solution['ME2'])

0.0
0.5416178638638556
4.354100850130325


In [45]:
old_DM25_cit_plus_model.reactions.OAADC.x

0.0

In [18]:
print(new_cit_minus_solution['MAEA'])
print(new_DM25_cit_plus_solution['MAEA'])
print(new_DM0_cit_plus_solution['MAEA'])

0.0
0.0
0.0


In [19]:
''' look at pyruvate in the old models'''
print(old_cit_minus_model.metabolites.pyr_c.summary())
print('************************')
print(old_DM25_cit_plus_model.metabolites.pyr_c.summary())
print('************************')
print(old_DM0_cit_plus_model.metabolites.pyr_c.summary())

                        PERCENT  FLUX                   REACTION_STRING                  
RXN_STAT  ID                                                                             
PRODUCING GLCptspp       79.6        1                 glc__D_p + pep_c --> g6p_c + pyr_c
          PYK            19.4    0.244              adp_c + h_c + pep_c --> atp_c + pyr_c
          CYSTL          1.06   0.0133    cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c
CONSUMING PDH            78.7    0.989  coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...
          ACLS             12    0.151              h_c + 2.0 pyr_c --> alac__S_c + co2_c
          VPAMTr_copy1      4   0.0503             3mob_c + ala__L_c <=> pyr_c + val__L_c
          DHDPS          2.55   0.0321     aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c
          ACHBS             2   0.0251         2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c
************************
                        PERCENT  FLUX                   REACTION_STRING    

cobra/core/summary/summary.py:206 UserWarning: Setting float_format to anything other than None will cause nan to be present in the output.


                        PERCENT  FLUX                   REACTION_STRING                  
RXN_STAT  ID                                                                             
PRODUCING ME2            97.5     4.35      mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
          CYSTL          2.48    0.111    cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c
CONSUMING PDH            49.7     2.22  coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...
          ACLS           28.1     1.26              h_c + 2.0 pyr_c --> alac__S_c + co2_c
          VPAMTr_copy1   9.35    0.418             3mob_c + ala__L_c <=> pyr_c + val__L_c
          DHDPS          5.97    0.266     aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c
          ACHBS          4.67    0.209         2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c
          PFL            1.76   0.0785                  coa_c + pyr_c --> accoa_c + for_c


In [20]:
''' look at pyruvate in the models'''
print(new_cit_minus_model.metabolites.pyr_c.summary())
print('************************')
print(new_DM25_cit_plus_model.metabolites.pyr_c.summary())
print('************************')
print(new_DM0_cit_plus_model.metabolites.pyr_c.summary())

                        PERCENT  FLUX                   REACTION_STRING                  
RXN_STAT  ID                                                                             
PRODUCING GLCptspp       79.6        1                 glc__D_p + pep_c --> g6p_c + pyr_c
          DHAPT          19.4    0.244                   dha_c + pep_c --> dhap_c + pyr_c
          CYSTL          1.06   0.0133    cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c
CONSUMING PDH            78.7    0.989  coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...
          ACLS             12    0.151              h_c + 2.0 pyr_c --> alac__S_c + co2_c
          VPAMTr_copy1      4   0.0503             3mob_c + ala__L_c <=> pyr_c + val__L_c
          DHDPS          2.55   0.0321     aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c
          ACHBS             2   0.0251         2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c
************************
                        PERCENT  FLUX                   REACTION_STRING    

In [21]:
''' let us look for reactions which increase flux in response to higher glucose input,
    but decrease flux in response to higher citrate input, or vice-versa.
    These would be candidate reactions for antagonistic pleiotropy.'''

## increment glucose or citrate influx from 10.0 to 11.0.
## note that result is somewhat sensitive to +/- epsilon: MAEA is pleiotropic for -e but not +e.
epsilon = 0.01

glucose_model_dx = new_cit_minus_model.copy()
citrate_model_dy = new_DM0_cit_plus_model.copy()
DM25_dx = new_cit_minus_model.medium
DM0_dy = new_DM0_cit_plus_model.medium
DM25_dx['EX_glc__D_e'] = DM25_dx['EX_glc__D_e'] - epsilon
DM0_dy['EX_cit_e'] = DM0_dy['EX_cit_e'] - epsilon
glucose_model_dx.medium = DM25_dx
citrate_model_dy.medium = DM0_dy
glucose_dx_solution = glucose_model_dx.optimize()
citrate_dy_solution = citrate_model_dy.optimize()

citrate_solution = new_DM0_cit_plus_model.optimize()
glucose_solution = new_cit_minus_model.optimize()

dCdy = citrate_dy_solution.fluxes - citrate_solution.fluxes
dGdx = glucose_dx_solution.fluxes - glucose_solution.fluxes

flux_derivative = pd.DataFrame({'dCdy':dCdy,'dGdx':dGdx})
flux_derivative['vec_norm'] = np.sqrt(np.square(flux_derivative).sum(axis=1))
flux_derivative['product'] = flux_derivative['dCdy']*flux_derivative['dGdx']

Read LP format model from file /tmp/tmpfn75j1a8.lp
Reading time = 0.01 seconds
: 1951 rows, 5498 columns, 21264 nonzeros
Read LP format model from file /tmp/tmplkjxamzx.lp
Reading time = 0.01 seconds
: 1951 rows, 5498 columns, 21264 nonzeros


In [22]:
flux_derivative2 = flux_derivative[flux_derivative['vec_norm'] > epsilon]

In [23]:
''' this result shows no tradeoff between glucose and citrate, neglecting the costs of gene expression.
    what if I fix oxaloacetate decarboxylation at some value? Perhaps some tradeoff. But gene regulation could
    alleviate the tradeoff.'''

''' my initial thinking/result was that maeA flux is sensitive to changes in glucose and citrate.
    But this finding does not seem to hold up-- this idea is cool so I keep the code, but "result"
    will not be presented.'''

dGdx_norm = np.sqrt(np.square(flux_derivative['dGdx']).sum())
dCdy_norm = np.sqrt(np.square(flux_derivative['dCdy']).sum())
theta = np.arccos(flux_derivative2['dCdy'].dot(flux_derivative2['dGdx'])/(dGdx_norm*dCdy_norm))
print(theta)
flux_derivative2.sort_values('vec_norm').round(2)

2.077488315028384


,dCdy,dGdx,vec_norm,product
EX_glc__D_e,0.00,0.01,0.01,0.00
GLCptspp,0.00,-0.01,0.01,-0.00
GLCtex_copy2,0.00,-0.01,0.01,-0.00
FUM,-0.01,-0.00,0.01,0.00
ICDHyr,-0.01,-0.00,0.01,0.00
...,...,...,...,...
ME2,-4.35,0.00,4.35,-0.00
SUCDi,18.50,-0.03,18.50,-0.63
NADH16pp,-18.52,0.00,18.52,-0.00
NADH17pp,18.51,-3.88,18.91,-71.79


In [24]:
''' look at acetyl-CoA in the old models'''
print(old_cit_minus_model.metabolites.accoa_c.summary())
print('************************')
print(old_DM25_cit_plus_model.metabolites.accoa_c.summary())
print('************************')
print(old_DM0_cit_plus_model.metabolites.accoa_c.summary())

                   PERCENT  FLUX                   REACTION_STRING                  
RXN_STAT  ID                                                                        
PRODUCING PDH       95.2    0.989  coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...
          ACS       4.85   0.0504   ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
CONSUMING CS        68.8    0.715    accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
          ACACT1r   5.92   0.0616                    2.0 accoa_c <=> aacoa_c + coa_c
          IPPS      3.75    0.039  3mob_c + accoa_c + h2o_c --> 3c3hmp_c + coa_c ...
          ACACT2r   2.96   0.0308             accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
          ACACT3r   2.96   0.0308             accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
          ACACT4r   2.96   0.0308             accoa_c + occoa_c <=> 3odcoa_c + coa_c
          ACACT5r   2.96   0.0308           accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
          ACGS      2.46   0.0256       accoa_c + glu__L_c --> ac

In [25]:
''' look at acetyl-CoA in the models'''
print(new_cit_minus_model.metabolites.accoa_c.summary())
print('************************')
print(new_DM25_cit_plus_model.metabolites.accoa_c.summary())
print('************************')
print(new_DM0_cit_plus_model.metabolites.accoa_c.summary())

                   PERCENT  FLUX                   REACTION_STRING                  
RXN_STAT  ID                                                                        
PRODUCING PDH       95.2    0.989  coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...
          PTAr      4.85   0.0504                  accoa_c + pi_c <=> actp_c + coa_c
CONSUMING CS        68.8    0.715    accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
          ACACT1r   5.92   0.0616                    2.0 accoa_c <=> aacoa_c + coa_c
          IPPS      3.75    0.039  3mob_c + accoa_c + h2o_c --> 3c3hmp_c + coa_c ...
          ACACT2r   2.96   0.0308             accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
          ACACT3r   2.96   0.0308             accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
          ACACT4r   2.96   0.0308             accoa_c + occoa_c <=> 3odcoa_c + coa_c
          ACACT5r   2.96   0.0308           accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
          ACGS      2.46   0.0256       accoa_c + glu__L_c --> ac